In [ ]:
make_data <- FALSE

O nosso objetivo é de prever o número de *sets* que um jogo de ténis profissional de Austrália vai ter, baseado num número de recursos que um jogo vai ter. 

Um jogo de ténis é composto por pontos, jogos e sets. De acordo com as [regras de ténis da Federação Internacional de Tennis (IFT)](https://www.atptour.com/en/corporate/rulebook), um set é uma pontuação, onde o primeiro jogador a ganhar 6 jogos ganha um set, sendo um jogo decidido pelo número de sets que um jogador ganha. Um jogo pode ser à melhor de 3 ou à melhor de 5, ou seja, o jogador precisa de ganhar 2 sets (o que traduz para 3 sets de máximo) ou 3 sets (máximo 5), respectivamente.  O jogador que decide o lado que começa/quem serve primeiro é decidido aleatoriamente, sendo que o outro decide a outra decisão.

Baseado nestes factos, parece que o projeto vai ser de regressão ordinal, sendo que o número de sets varia entre 2 e 5, não mais e não menos.

# Raw

In [ ]:
if (make_data){
    source(here("scripts", "toParquet.R"))
}
arrow::read_parquet(here("data", "atpRaw.parquet"), as_data_frame = TRUE) -> atp
head(atp)

,Born,Date,GameRank,GameRound,Ground,Hand,Height,LinkPlayer,Location,Oponent,PlayerName,Prize,Score,Tournament,WL,_id.$oid
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,"Belgrade, Serbia",2021.11.22 - 2021.12.05,30,Semi-Finals,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,"Madrid, Spain",Marin Cilic,Novak Djokovic,,64 62,Davis Cup Finals,W,624ab34913b144c54b3c9aba
2,"Belgrade, Serbia",2022.02.21 - 2022.02.26,26,Round of 16,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,"Dubai, U.A.E.",Karen Khachanov,Novak Djokovic,"$2,794,840",63 76,Dubai,W,624ab34913b144c54b3c9abb
3,"Belgrade, Serbia",2021.11.15 - 2021.11.21,12,Round Robin,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,"Turin, Italy",Cameron Norrie,Novak Djokovic,"$7,250,000",62 61,Nitto ATP Finals,W,624ab34913b144c54b3c9abc
4,"Belgrade, Serbia",2021.11.15 - 2021.11.21,5,Round Robin,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,"Turin, Italy",Andrey Rublev,Novak Djokovic,"$7,250,000",63 62,Nitto ATP Finals,W,624ab34913b144c54b3c9abd
5,"Belgrade, Serbia",2021.11.01 - 2021.11.07,2,Finals,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,"Paris, France",Daniil Medvedev,Novak Djokovic,"�5,207,405",46 63 63,ATP Masters 1000 Paris,W,624ab34913b144c54b3c9abe
6,"Belgrade, Serbia",2021.11.01 - 2021.11.07,10,Semi-Finals,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,"Paris, France",Hubert Hurkacz,Novak Djokovic,"�5,207,405",36 60 76,ATP Masters 1000 Paris,W,624ab34913b144c54b3c9abf


In [ ]:
nrow(atp)

[1] 1308835

A base de dados mostra informação pública de [ATP Tour](https://www.atptour.com/en/players).

A primeira vista dos dados revela vários factos:

* A base de dados tem 1308835, sendo cada registro um jogo de um jogador "principal" (com as informações de tal) contra um oponente (sem as informações de tal).
    * Isto poderá indicar que teremos de transformar os dados em registros de jogos entre os dois jogadores, sem jogador principal. Não temos a certeza se isto é necessário.
* Ao fazer a cabeça dos dados, parece que apenas mostra o jogador Novak Djokovic como principal. Isto pode indicar que a base de dados pode estar ordenada de uma certa forma.
    * Esta ordem apenas pode indicar como os dados foram obtidos, e nada sobre um registro específico. Informação sobre a recolha pode ajudar a perceber melhor o contexto da base de dados.
* A data parece mostrar as datas do torneio e não do jogo especificamente
* A coluna `prize` parece mal formatada, com valores omissos presentes. Da mesma forma, aprece que os valores estão em várias moedas diferentes.
* A coluna `GameRank` parece indicar o *rank* do oponente, não o rank do jogo.
    * Desta forma, desde que o oponente apareça também como principal, consegue-se ter o rank de ambas as pessoas. No entanto, isto não é garantido.
    * Nos casos que isto acontece, pode ajudar fazer a diferença de rank entre jogadores.
* A coluna `Hand` parece incluir ambas as informações do *backhand* e se é destro ou esquerdino
* O score parece conter a pontuação de cada set, sendo ai onde vamos buscar o número de sets.

In [ ]:
set.seed(1)
atp %>% slice_sample(n=5)

Born,Date,GameRank,GameRound,Ground,Hand,Height,LinkPlayer,Location,Oponent,PlayerName,Prize,Score,Tournament,WL,_id.$oid
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
,2010.08.02 - 2010.08.08,-,Semi-Finals,Clay,null,NA,https://www.atptour.com/en/players/juan-lizariturry/l996/player-activity?year=all&matchType=Singles,Spain,Taro Daniel,Juan Lizariturry,"$10,000",63 62,Spain F28,W,624ab365a5c952847318a12b
"Sendai, Japan",2017.03.13 - 2017.03.19,164,Round of 16,Hard,"Right-Handed, Two-Handed Backhand",175,https://www.atptour.com/en/players/yuichi-sugita/se73/player-activity?year=all&matchType=Singles,"Shenzhen, China",Jan Satral,Yuichi Sugita,"$75,000","67, 63 62",Shenzhen,W,624ab34f936e4d97ffee0700
,2005.11.21 - 2005.11.27,1367,Round of 32,Hard,null,NA,https://www.atptour.com/en/players/ludwig-pellerin/pa58/player-activity?year=all&matchType=Singles,"Monastin, Tunisia",Anas Fattar,Ludwig Pellerin,"$10,000",64 64,Tunisia F5,W,624ab40fba7043df73e7f9e9
"Voorhees, NJ, USA",2015.04.20 - 2015.04.26,462,2nd Round Qualifying,Clay,"Right-Handed, Two-Handed Backhand",185,https://www.atptour.com/en/players/tommy-paul/pl56/player-activity?year=all&matchType=Singles,"Savannah, GA, U.S.A.",Andrea Collarini,Tommy Paul,"$50,000","76, 62",Savannah,W,624ab34a13b144c54b3cfd0e
"Santiago, Chile",2007.03.19 - 2007.04.01,107,1st Round Qualifying,Hard,"Right-Handed, Two-Handed Backhand",191,https://www.atptour.com/en/players/paul-capdeville/c634/player-activity?year=all&matchType=Singles,"Miami, FL, U.S.A.",Andrei Pavel,Paul Capdeville,"$3,200,000",26 63 75,ATP Masters 1000 Miami,W,624ab37573539d8f27174222


Estes dados mostram também que há jogadores principais sem informações.

In [ ]:
atp %>% distinct(PlayerName)

PlayerName
<chr>
Novak Djokovic
Daniil Medvedev
Rafael Nadal
Alexander Zverev
Stefanos Tsitsipas
Matteo Berrettini
Andrey Rublev
Casper Ruud
Felix Auger-Aliassime


## Paises
Nós pudemos observar no projeto anterior que a limpeza de dados geográficos podem levar a problemas. Decidimos pegar nos dados previamente feitos par a limpeza destes, de forma a garantir que estes se encontram consistentes para análise futura e para a seleção de torneiros australianos apenas e na sua totalidade.

In [ ]:
if (make_data) {
    source(here("scripts", "locations.R"))
}
atpC <- arrow::read_parquet(here("data", "atpCountries.parquet"))
head(atpC)

Date,GameRank,GameRound,Ground,Hand,Height,LinkPlayer,Oponent,PlayerName,Prize,Score,Tournament,WL,_id.$oid,tournament.location,principal.born
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2021.11.22 - 2021.12.05,30,Semi-Finals,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Marin Cilic,Novak Djokovic,,64 62,Davis Cup Finals,W,624ab34913b144c54b3c9aba,Spain,Spain
2022.02.21 - 2022.02.26,26,Round of 16,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Karen Khachanov,Novak Djokovic,"$2,794,840",63 76,Dubai,W,624ab34913b144c54b3c9abb,United Arab Emirates,United Arab Emirates
2021.11.15 - 2021.11.21,12,Round Robin,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Cameron Norrie,Novak Djokovic,"$7,250,000",62 61,Nitto ATP Finals,W,624ab34913b144c54b3c9abc,Italy,Italy
2021.11.15 - 2021.11.21,5,Round Robin,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Andrey Rublev,Novak Djokovic,"$7,250,000",63 62,Nitto ATP Finals,W,624ab34913b144c54b3c9abd,Italy,Italy
2021.11.01 - 2021.11.07,2,Finals,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Daniil Medvedev,Novak Djokovic,"�5,207,405",46 63 63,ATP Masters 1000 Paris,W,624ab34913b144c54b3c9abe,France,France
2021.11.01 - 2021.11.07,10,Semi-Finals,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Hubert Hurkacz,Novak Djokovic,"�5,207,405",36 60 76,ATP Masters 1000 Paris,W,624ab34913b144c54b3c9abf,France,France


## Date

In [ ]:
atpC %>% 
    separate_wider_delim(Date, " - ", names=c("ds", "df"), too_few = "debug") %>%
    select(ds, df, Date, Date_ok, Date_pieces, Date_remainder) -> atpCD.debug

Warning message:
"Debug mode activated: adding variables `Date_ok`, `Date_pieces`, and `Date_remainder`."


In [ ]:
atpCD.debug %>% filter(Date_ok == F)

ds,df,Date,Date_ok,Date_pieces,Date_remainder
<chr>,<chr>,<chr>,<lgl>,<int>,<chr>
2011.05.09,NA,2011.05.09,FALSE,1,
2011.05.09,NA,2011.05.09,FALSE,1,
2011.05.09,NA,2011.05.09,FALSE,1,
2011.05.09,NA,2011.05.09,FALSE,1,
2011.11.21,NA,2011.11.21,FALSE,1,
2011.11.21,NA,2011.11.21,FALSE,1,
2011.11.21,NA,2011.11.21,FALSE,1,
2011.11.21,NA,2011.11.21,FALSE,1,
2011.11.21,NA,2011.11.21,FALSE,1,


In [ ]:
# adapted from sgutreuter/SGmisc
#mid_date <- function(start, end){
#    inter <- interval(start, end)
#    res <- int_start(inter) + ((int_end(inter) - int_start(inter)) / 2)
#    as_date(res)
#}

In [ ]:
atpCD <- atpC %>% 
    separate_wider_delim(Date, " - ", names=c("ds", "df"), too_few = "align_start") %>% 
    mutate(tournament.date_start = ymd(ds), tournament.date_finish = ymd(df)) %>%
    select(-ds, -df) %>%
    mutate(tournament.duration = ifelse(
        tournament.date_finish %>% is.na(),
        0,
        tournament.date_finish - tournament.date_start
    ))#, tournament.midDay = ifelse(
     #   tournament.date_finish %>% is.na(),
     #   tournament.date_start,
     #   mid_date(tournament.date_start, tournament.date_finish)
     #)

In [ ]:
atpCD

GameRank,GameRound,Ground,Hand,Height,LinkPlayer,Oponent,PlayerName,Prize,Score,Tournament,WL,_id.$oid,tournament.location,principal.born,tournament.date_start,tournament.date_finish,tournament.duration
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<date>,<dbl>
30,Semi-Finals,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Marin Cilic,Novak Djokovic,,64 62,Davis Cup Finals,W,624ab34913b144c54b3c9aba,Spain,Spain,2021-11-22,2021-12-05,13
26,Round of 16,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Karen Khachanov,Novak Djokovic,"$2,794,840",63 76,Dubai,W,624ab34913b144c54b3c9abb,United Arab Emirates,United Arab Emirates,2022-02-21,2022-02-26,5
12,Round Robin,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Cameron Norrie,Novak Djokovic,"$7,250,000",62 61,Nitto ATP Finals,W,624ab34913b144c54b3c9abc,Italy,Italy,2021-11-15,2021-11-21,6
5,Round Robin,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Andrey Rublev,Novak Djokovic,"$7,250,000",63 62,Nitto ATP Finals,W,624ab34913b144c54b3c9abd,Italy,Italy,2021-11-15,2021-11-21,6
2,Finals,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Daniil Medvedev,Novak Djokovic,"�5,207,405",46 63 63,ATP Masters 1000 Paris,W,624ab34913b144c54b3c9abe,France,France,2021-11-01,2021-11-07,6
10,Semi-Finals,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Hubert Hurkacz,Novak Djokovic,"�5,207,405",36 60 76,ATP Masters 1000 Paris,W,624ab34913b144c54b3c9abf,France,France,2021-11-01,2021-11-07,6
8,Round Robin,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Casper Ruud,Novak Djokovic,"$7,250,000","76, 62",Nitto ATP Finals,W,624ab34913b144c54b3c9ac0,Italy,Italy,2021-11-15,2021-11-21,6
118,Round Robin,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Dennis Novak,Novak Djokovic,,63 62,Davis Cup Finals,W,624ab34913b144c54b3c9ac1,Spain,Spain,2021-11-22,2021-12-05,13
123,Quarter-Finals,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Jiri Vesely,Novak Djokovic,"$2,794,840",46 67,Dubai,L,624ab34913b144c54b3c9ac2,United Arab Emirates,United Arab Emirates,2022-02-21,2022-02-26,5


In [ ]:
options(scipen=999)
atpCD %>% group_by(tournament.duration) %>% summarise(n=n(), freq = n/nrow(atpCD)*100) %>% arrange(-n)

tournament.duration,n,freq
<dbl>,<int>,<dbl>
6,1144856,87.4713772171
13,118095,9.0229096869
5,29228,2.2331309905
0,12445,0.9508455993
7,1672,0.1277471950
14,988,0.0754869789
20,306,0.0233795704
9,236,0.0180313026
12,231,0.0176492835


TODO analise disto